# Counting Boats from Space (Part 1): Annotate Chips with Boat Counts

In [ ]:
%reload_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv
%matplotlib inline

## 1. Import requirements

In [ ]:
import os 
import sys
import json
import shapely.geometry
import IPython.display
from xcube_sh.cube import open_cube
from xcube_sh.observers import Observers

In [ ]:
sys.path.insert(0,os.path.dirname('../src/'))

In [ ]:
from GIS_utils import bbox_from_point
from config import CubeConfig
from preprocess import preprocess, plot_cube_and_background, save_labels

In [ ]:
data_dir = "/home/jovyan/data" # data directory (path to save chips)
#RADIUS = 500 # AOI radius in meters

## 2. Specify AOI + time window

In [ ]:
with open('../data/aoi.json', 'r') as f:
    aoi_file = json.load(f)
    coords = aoi_file['Final'] ### Choose AOI set (Straits, Tourism, etc.)
    aoi_idx = 0  ### Choose location index
    aoi_name = list(coords.keys())[aoi_idx]
    lat, lon, RADIUS = coords[aoi_name][0][0], coords[aoi_name][0][1], coords[aoi_name][0][2]
    #lat, lon, RADIUS = 8.87, -79.535, 3000 change lat, lon, RADIUS
    print('{}/{} {}: {}, {}, {}'.format(aoi_idx+1, len(coords), aoi_name, lat, lon, RADIUS))

In [ ]:
start_date = '2020-03-22'
end_date = '2020-06-21'

In [ ]:
bbox = bbox_from_point(lat=lat, lon=lon, r=RADIUS) # WGS84 coordinates
IPython.display.GeoJSON(shapely.geometry.box(*bbox).__geo_interface__)

## 3. Load the data in a cube

In [ ]:
cube_config = CubeConfig(dataset_name='S2L1C', band_names=['B03', 'B08', 'CLP'], # GREEN + NIR + Clouds
                         tile_size=[2*RADIUS//10, 2*RADIUS//10], geometry=bbox,
                         time_range=[start_date, end_date], time_period='1D')

In [ ]:
request_collector = Observers.request_collector()
#request_collector.stats # statistics for SentinelHub request

In [ ]:
cube = open_cube(cube_config, observer=request_collector)
#cube # Verify the content of the cube

## 4. Preprocess Cube

In [ ]:
cube, background_ndwi = preprocess(cube, max_cloud_proba=0.3, nans_how='any', verbose=1, plot_NDWI=True)
#cube

In [ ]:
plot_cube_and_background(cube, background_ndwi, t=-1) # display results for each band

## 5. Annotate imagery + save results (B03, B08, CLP, bg_NDWI)

In [ ]:
label = input().split(',') # comma value separated
label = [int(l) for l in label]
assert len(label) == len(cube.time)
save_labels(cube, background_ndwi, label, lat_lon=(lat,lon), data_dir=os.path.join(data_dir, 'chips'), label_filename='../data/labels.csv')